In [0]:
%fs ls dbfs:/FileStore/tables/


In [0]:
%python
dbutils.fs.ls("/FileStore/tables/")

In [0]:
show tables;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore

In [0]:
select * from books;

In [0]:
drop table books;

In [0]:
show tables;

In [0]:
describe extended bronze_table;

In [0]:
show views;

In [0]:
create or replace temp view books_new_1
(book_id string, book_name string, author string, category string, cost double)
using csv
options(
  delimiter = ';',
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv/',
  header = 'true'
);

In [0]:
select * from books_new_1;

In [0]:
create table books_new_csv as 
select * from books_new_1;

In [0]:
%python
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .appName("MySparkApplication") \
                    .getOrCreate()

In [0]:
%python
(spark.readStream
        .table("books_new_csv")
        .createOrReplaceTempView("Books_Streaming_View"));

In [0]:
-- drop view books_streaming_view;

In [0]:
select * from books_streaming_view;

In [0]:
select author, count(book_id) as total_books
from books_streaming_view
group by author;

In [0]:
drop view author_counts_tmp_view;

In [0]:
create or replace Temp view author_counts_tmp_view as (
  select author, count(book_id) as total_books
  from books_streaming_view
  group by author
);

In [0]:
select * from author_counts_tmp_view; 

In [0]:
%python
(spark.table("author_counts_tmp_view")
      .writeStream
      .trigger(processingTime='4 seconds')
      .outputMode("complete")
      .option("checkpointLocation","dbfs:/mnt/demo/author_counts_checkpoint")
      .table("author_counts_2"))

In [0]:
select * from author_counts_2;

In [0]:
INSERT INTO books
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
         ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
         ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35);

In [0]:
select * from books;

In [0]:
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)

In [0]:
%python
(spark.table("author_counts_tmp_view")
      .writeStream
      .trigger(processingTime='4 seconds')
      .outputMode("complete")
      .option("checkpointLocation","dbfs:/mnt/demo/author_counts_checkpoint")
      .table("author_counts")
    #   .awaitTermination()
      )

In [0]:
select * from author_counts_1;